In [ ]:
import numpy as np
import itertools
import gurobipy as gpy
import whalrus

import nbimporter
import m00_helper as helper
import m01_profiles as profiles

In [ ]:
def random_winners(n_cands, committee_size, seed=None):
    np.random.seed(seed)
    return np.random.choice(np.arange(n_cands), committee_size, replace=False)

#### Rules based on preference orders/ordermaps

In [ ]:
def scoring_ordermaps(ordermap, score_vector):
    '''
    Given an ordermap and a score_vector, return a vector of the scores of the candidates
    
    PARAMS:
    ordermap (np.ndarray): 2D matrix of size n_voters x n_cands. 
                            The value ordermap[i,j] is the rank voter i gives to cand j
    score_vector (np.ndarray): 1D array of numbers of score increase a cand gets based on the rank a voter gives them
    
    NOTES:
    Currently using nested for loops which is very slow but easy to read and debug
    '''
    scores = np.zeros_like(score_vector)
    for row in ordermap:
        for cand,rank in enumerate(row):
            scores[cand] += score_vector[rank]
    return scores

In [ ]:
def scoring_rule_ordermap(ordermap, score_vector, committee_size, seed=None):
    scores = scoring_ordermaps(ordermap, score_vector)
    augmented_scores = helper.array1D_to_sorted_indices(scores, seed)
    winners = np.flip(augmented_scores[:,2][-committee_size:].astype(int))
    return winners, scores

In [ ]:
def plurality(ordermap, committee_size, seed=None):
    score_vector = np.zeros(ordermap.shape[1], dtype=int)
    score_vector[0] += 1
    return scoring_rule_ordermap(ordermap, score_vector, committee_size, seed)

In [ ]:
def borda(ordermap, committee_size, seed=None):
    n_cands = ordermap.shape[1]
    score_vector = np.arange(n_cands-1, -1, -1, dtype=int)
    return scoring_rule_ordermap(ordermap, score_vector, committee_size, seed)

In [ ]:
def scoring_rule_old(score_vector, ranks, candidates, committee_size, seed=None):
    '''
    Parameters
    ----------
    score_vector: vector of numbers of len candidates that defines how much weight to give to that position.
    
    ranks: dict of voters to a ranked list of candidates (candidate list can be incomplete.)
    
    candidates: list of all candidates.
    
    committee_size: int < candidates that is the size of the comittee.  Return in ranked order.
    
    Returns
    -------
    committee: list of candidates in order of their STV elimination, result
    
    Notes
    -----
    This mereley truncates the scoring rule if votes are incomplete.. there is an axiomatic argument for doing an
    average.
    
    '''
    result = {x:0.0 for x in candidates}
    for v_id,v_pref in ranks.items():
        for i,c in enumerate(v_pref):
            result[c] += score_vector[i]
    sorted_list = [c for c in sorted(result, key=result.get, reverse=True)]
    
    #Break ties randomly between cands with same score
    tiebreakers = helper.create_tiebreakers(len(sorted_list), seed, dtype=int)
    array_augmented = np.column_stack((sorted_list, tiebreakers))
    sorted_indices = np.lexsort((array_augmented[:, 1], array_augmented[:, 0]))
    
    return sorted_indices[:committee_size], result

In [ ]:
def single_transferable_vote(orders, committee_size, seed=None):
    '''
    PARAMETERS
    ----------
    orders: np.ndarray with a row for each voter that is a ranked list of candidates
    
    candidates: list of all candidates.
    
    committee_size: int < candidates that is the size of the comittee.  Return in ranked order.
    
    RETURNS
    -------
    committee: list of candidates in order of their STV elimination.
    
    
    NOTES
    ------
    orders matrix is currently converted to dict to take advantage of old implementation. Slow but it works.
    
    '''
    # Compute the STV rank.. that is keep deleting candidates till we have no one left...
    unranked_cands = list(range(orders.shape[1]))
    ranks = {voter:ranking for voter,ranking in enumerate(orders)}
    modified_ranks = copy.copy(ranks)
    result = []
    while len(unranked_cands) > 0:
        # Compute the current plurality score of all candidates.
        c, plurality_scores = scoring_rule_old([1] + [0]*(len(unranked_cands)-1), ranks, unranked_cands, len(unranked_cands))
        #print("plurality order:",str(c))
        #print("plurl scores",str(plurality_scores))
        # Delete the lowest and add him to the list.
        result.insert(0,c[-1])
        #print(result)
        #print(unranked_cands)
        unranked_cands.remove(c[-1])
        # Need to delete him from all votes...
        #print(modified_ranks)
        # remove the candidate.
        for i,p in modified_ranks.items():
            p.remove(c[-1])
        #print(modified_ranks)
    return result[:committee_size]

#### Approval-based rules

In [ ]:
def max_approval(approvals_profile, committee_size, seed=None):
    approval_counts = np.sum(approvals_profile, axis=0)
    augmented_counts = helper.array1D_to_sorted_indices(approval_counts, seed)
    return augmented_counts[:,2][-committee_size:]

In [ ]:
def repeated_alternative_vote(approvals, committee_size):
    '''
    Parameters
    ----------
    approvals: voter indexed list of approval ballots.
    
    candidates: list of candidates.
    
    committee_size: int < candidates that is the size of the comittee.  Return in ranked order that the committee was selected.
    
    Returns
    -------
    committee: list of candidates in order of their STV elimination, result
    
    Notes
    -----
    Runs RAV in a fairly efficient way...
    
    '''
    candidates = list(range(approvals.shape[1]))
    result = []
    while len(result) < committee_size:
        # Select the winner weighted 1/approval set intersection.
        non_winners = set(candidates) - set(result)
        c_scores = {c:0 for c in non_winners}
        
        for v_id, v_pref in approvals.items():
            # every non-winning approved candidate get 1/intersection points..
            intersect = len(set(result).intersection(v_pref)) + 1
            for c in non_winners:
                if c in v_pref:
                    c_scores[c] += 1.0 / intersect
        # choose the highest score...
        #print(c_scores)
        order = [k for k in sorted(c_scores, key=c_scores.get, reverse=True)]
        
        tiebreakers = helper.create_tiebreakers(len(order), dtype=int)
        array_augmented = np.column_stack((order, tiebreakers))
        order_tiebroken = np.lexsort((array_augmented[:, 1], array_augmented[:, 0]))
        
        result.append(order_tiebroken[0])
    return result

# voters_on_issues, candidates_on_issues = genpref.create_binary_random_preferences(5, 5, 5)
# print("Voters: " + str(voters_on_issues))
# print("Cands: " + str(candidates_on_issues))
# approvals = issues_to_approvals(voters_on_issues, candidates_on_issues)
# print(approvals)
# print(repeated_alternative_vote(approvals, candidates_on_issues.keys(), 3))

#### Score-based rules

In [ ]:
def max_score(scores_profile, committee_size, seed=None):
    scores = np.sum(scores_profile, axis=0)
    augmented_scores = helper.array1D_to_sorted_indices(scores, seed)
    scores_tiebroken = augmented_scores[:,0]
    winners = augmented_counts[:,2][-committee_size:]
    return winners, scores_tiebroken

### Convenience Functions

In [ ]:
def all_election_rules():
    return random_winners, plurality, borda, single_transferable_vote, 

### Tests

In [ ]:
voter_profile = profiles.issues_profile(3, 3, p=0.5)
cands_profile = profiles.issues_profile(3, 3, p=0.5)
print(f'voter profile:\n{voter_profile}')
print(f'cands profile:\n{voter_profile}')
print(cands_profile)
distances = profiles.issues_to_distances(voter_profile, cands_profile)
print(f'distances:\n{distances}')
ordermaps = profiles.distances_to_ordermaps(distances)
print(f'ordermaps:\n{ordermaps}')
plurality_winners, plurality_scores = plurality(ordermaps, 2)
print(f'plurality winners: {plurality_winners}')
print(f'plurality scores: {plurality_scores}')
borda_winners, borda_scores = borda(ordermaps, 2)
print(f'borda winners: {borda_winners}')
print(f'borda scores: {borda_scores}')

voter profile:
[[0 1 0]
 [0 0 0]
 [1 1 0]]
cands profile:
[[0 1 0]
 [0 0 0]
 [1 1 0]]
[[0 0 0]
 [0 1 0]
 [0 0 1]]
distances:
[[0.33333333 0.         0.66666667]
 [0.         0.33333333 0.33333333]
 [0.66666667 0.33333333 1.        ]]
ordermaps:
[[1 0 2]
 [0 1 2]
 [1 0 2]]
plurality winners: [1 0]
plurality scores: [1 2 0]
borda winners: [1 0]
borda scores: [4 5 0]


### Notes

If the score of a cand from an agent is 1-Hamming Distance, then isn't taking the top cands with the highest sum of scores the same as taking the top cands with the lowest sum of distances? This seems like a greedy way of selecting k clustering centers, as comapred to k-median. So this difference may be worth noting.

# TO DO

#### NP-Hard Rules based on OWAs

In [ ]:
def chamberlin_courant(voter_issue_profile, cands_issue_profile, committee_size, seed=None):
    return

In [ ]:
 ###Q: What type of profile does this function take? Approvals, Rankings, or Scores
def k_median(profile, committee_size):
    return